##  Butterfly Images - Pytorch

In June 2016, I read an article about tensorflow practiced on butterfly images. Just right after presenting my Master's capstone project which I changed from Convulutional Neural Networks to Enterprise Architecture. I didn't know Neural Networks were the building block of Deep Learning or the blog I read is about Deep Learning and Computer Vision. This article triggered to start my learning journey on Deep Learning, Computer Vision and Artificial Intelligence. You can find the blog post here:  [a poet does tensorflow](https://www.oreilly.com/learning/a-poet-does-tensorflow) 


In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

```dataset = datasets.ImageFolder('path/to/data', transform=transform)```

## Load Data

In [2]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of data set to use as test
test_size = 0.2


transform = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


data_set = datasets.ImageFolder(root="data",transform=transform)
dataloader = torch.utils.data.DataLoader(data_set, batch_size=4,shuffle=True,num_workers=2)

# obtain training indices that will be used for test
num_data = len(data_set)
indices = list(range(num_data))
np.random.shuffle(indices)
split = int(np.floor(test_size * num_data))
train_idx, test_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
test_sampler  = SubsetRandomSampler(test_idx)

# prepare data loaders
trainloader = torch.utils.data.DataLoader(data_set, batch_size=batch_size,
                                           sampler = train_sampler, num_workers=num_workers)
testloader  = torch.utils.data.DataLoader(data_set, batch_size=batch_size, 
                                           sampler = test_sampler, num_workers=num_workers)

classes = ('blurry','clear')


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Train Images
Here we are defining our network architecture and training them

In [5]:
model = nn.Sequential(nn.Linear(150528, 500),
                      nn.Linear(500, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

epochs = 30
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten images into a long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: 1.0611225727832678
Training loss: 0.6043970593900392
Training loss: 0.5429921357920675
Training loss: 0.5433650233528831
Training loss: 0.5538575423486305
Training loss: 0.5491892677364927
Training loss: 0.5382337434725328
Training loss: 0.5180408950104858
Training loss: 0.5232874136982542
Training loss: 0.49697643518447876
Training loss: 0.4888196596593568
Training loss: 0.5189437491424156
Training loss: 0.48548668442350446
Training loss: 0.507999472545855
Training loss: 0.49990487121271365
Training loss: 0.4926995291854396
Training loss: 0.5013617800943779
Training loss: 0.4933172852704019
Training loss: 0.500824690768213
Training loss: 0.48462424359538336
Training loss: 0.4833958266359387
Training loss: 0.47013867939963483
Training loss: 0.49537186911611847
Training loss: 0.49338152462785895
Training loss: 0.4898672717990297
Training loss: 0.49308923157778656
Training loss: 0.5086138898676092
Training loss: 0.4916872644063198
Training loss: 0.4727043920394146
Training

In [6]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [7]:
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

odict_keys(['0.weight', '0.bias', '1.weight', '1.bias', '3.weight', '3.bias', '5.weight', '5.bias'])


In [8]:
model.load_state_dict(state_dict)

## Transfer Learning

For  transfer learning let's use one of the most famous pre-trained models:resnet.

In [9]:
from torchvision import models

model_resnet152 = models.resnet152(pretrained=True)
model_resnet152

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [10]:
# Freeze parameters so we don't backprop through them
for param in model_resnet152.parameters():
    param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model_resnet152.classifier = classifier

optimizer = optim.SGD(model_resnet152.parameters(), lr=0.003)
criterion = nn.NLLLoss()

In [11]:
epochs = 30
steps = 0
running_loss = 0
print_every = 5
device = 'cpu'
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model_resnet152.forward(inputs)
        loss = criterion(logps, labels)
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model_resnet152.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model_resnet152.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model_resnet152.train()

Epoch 1/30.. Train loss: 0.960.. Test loss: 1.205.. Test accuracy: 0.000
Epoch 1/30.. Train loss: 0.789.. Test loss: 1.365.. Test accuracy: 0.000
Epoch 1/30.. Train loss: 0.834.. Test loss: 1.181.. Test accuracy: 0.006
Epoch 1/30.. Train loss: 0.733.. Test loss: 1.362.. Test accuracy: 0.000
Epoch 1/30.. Train loss: 0.857.. Test loss: 1.361.. Test accuracy: 0.000
Epoch 1/30.. Train loss: 0.753.. Test loss: 1.306.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.938.. Test loss: 1.066.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.818.. Test loss: 1.023.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.824.. Test loss: 0.983.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.877.. Test loss: 1.005.. Test accuracy: 0.006
Epoch 2/30.. Train loss: 0.887.. Test loss: 1.188.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.867.. Test loss: 1.137.. Test accuracy: 0.000
Epoch 2/30.. Train loss: 0.790.. Test loss: 1.131.. Test accuracy: 0.000
Epoch 3/30.. Train loss: 0.703.. Test loss: 1.234..

Epoch 18/30.. Train loss: 0.713.. Test loss: 0.589.. Test accuracy: 0.000
Epoch 18/30.. Train loss: 1.014.. Test loss: 0.826.. Test accuracy: 0.000
Epoch 18/30.. Train loss: 1.011.. Test loss: 1.098.. Test accuracy: 0.000
Epoch 18/30.. Train loss: 0.733.. Test loss: 1.167.. Test accuracy: 0.000
Epoch 18/30.. Train loss: 0.743.. Test loss: 0.981.. Test accuracy: 0.000
Epoch 18/30.. Train loss: 0.809.. Test loss: 1.083.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.733.. Test loss: 1.435.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.927.. Test loss: 0.933.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.813.. Test loss: 1.116.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.829.. Test loss: 0.970.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.775.. Test loss: 1.418.. Test accuracy: 0.006
Epoch 19/30.. Train loss: 0.970.. Test loss: 0.532.. Test accuracy: 0.000
Epoch 19/30.. Train loss: 0.715.. Test loss: 0.716.. Test accuracy: 0.000
Epoch 20/30.. Train loss: 0.754.. Test

In [12]:
torch.save(model_resnet152.state_dict(), 'resnet_checkpoint.pth')